In [1]:
import torch
from torch.autograd import Variable

import pyro
import pyro.distributions as dist

## Primitive stochastic functions

In [2]:
mu = Variable(torch.zeros(1))   # mean zero
sigma = Variable(torch.ones(1)) # unit variance
x = dist.normal(mu, sigma)

x

Variable containing:
 0.2865
[torch.FloatTensor of size 1]

In [4]:
log_p_x = dist.normal.log_pdf(x, mu, sigma)
log_p_x

Variable containing:
-0.9600
[torch.FloatTensor of size 1]

## The `pyro.sample` Primitive

In [12]:
# x will be called as 'my_sample' in pyro backend,
# and is a sample from dist.normal(mu, sigma)
x = pyro.sample("my_sample", dist.normal, mu, sigma)

x

Variable containing:
1.00000e-02 *
 -7.3809
[torch.FloatTensor of size 1]

# A simple Model

In [17]:
def weather():
    # cloudy is a random variable sampled from
    # Bernoulli distribution with p = 0.3
    # i.e. it is 30% cloudy and 70% sunny
    cloudy = pyro.sample('cloudy', dist.bernoulli,
                        Variable(torch.Tensor([0.3])))
    # given sampling result, cloudy,
    # assign corresponding string.
    # i.e. if Bernoulli trial succeeded, cloudy = 'cloudy'
    # else, cloudy = 'sunny'
    cloudy = 'cloudy' if cloudy.data[0] == 1.0 else 'sunny'
    # each weather has its own temperature distribution
    mean_temp = {'cloudy': [55.0], 'sunny': [75.0]}[cloudy]
    sigma_temp = {'cloudy': [10.0], 'sunny': [15.0]}[cloudy]
    # sample from temperature distribution
    temp = pyro.sample('temp', dist.normal,
                      Variable(torch.Tensor(mean_temp)),
                      Variable(torch.Tensor(sigma_temp)))
    
    return cloudy, temp.data[0]

In [18]:
for _ in range(3):
    print(weather())

('cloudy', 71.4061279296875)
('cloudy', 55.91141891479492)
('sunny', 75.79878234863281)


In [20]:
def ice_cream_sales():
    cloudy, temp = weather()  # reusing weather as a module.
    expected_sales = [200] if cloudy == 'sunny' and temp > 80.0 else [50]
    ice_cream = pyro.sample('ice_cream', dist.normal,
                           Variable(torch.Tensor(expected_sales)),
                           Variable(torch.Tensor([10.0])))
    return ice_cream